# Genes to Phenotypes and Phenotypes to Genes

This notebook allows you to query the gene and phenotype annotations in the IDR. 

**1. Gene to Phenotype queries**
You can query with a gene list or GO ontology ID for a protein complex/cellular component which is then queried for the associated gene members and get back phenotypes associated with those genes. 


**2. Phenotype to Gene queries**
You can query with a phenotype name and get back a list of all the genes linked 
to that phenotype.  You can then perform a GO enrichment analysis to see if the genes
are signficantly associated with any GO terms.




### First set up your environment and import all the packages you will need

In [1]:
# import pandas for data manipulation
import pandas

# import some functions for displaying the results
from ipywidgets import widgets, interact, fixed
from functools import wraps
from IPython.display import display, HTML

# import functions from the idr package to perform the queries
from idr import connection, create_http_session
from idr import genes_of_interest_go
from idr.widgets import textbox_widget, progress
from idr.widgets import dropdown_widget
from idr import get_phenotypes_for_genelist, get_similar_genes
from idr import attributes_by_attributes, get_organism_screenids

# import a package for doing GO term enrichement analysis
import gseapy as gp

### Create a session for querying the IDR

In [2]:
session = create_http_session()

### Set some variables to say that are going to only look at human genes

In [3]:
organism = 'Homo sapiens'
tax_id = '9606'


<br>
## Genes to Phenotypes query

### Create boxes to enter the Gene Ontology ID or manual gene list

In [4]:
go_term = textbox_widget('', 'Enter GO Id e.g. GO:0005885', 'Gene Ontology Id:', True)
manual_gene_list = textbox_widget('','Comma separated gene symbols', 'Manual Gene List:', True)

# ENTER VALUES IN ONE OR BOTH BOXES THEN MOVE TO THE NEXT CELL WITHOUT RUNNING THE CELL AGAIN

A Jupyter Widget

A Jupyter Widget

### Query GO for the genes associated with the GO ID, combine with any manually entered gene list

In [54]:
go_gene_list = []
if go_term.value.split(",") != ['']:
    go_gene_list = genes_of_interest_go(go_term.value, tax_id)
else:
    print 'Please enter a valid Gene Ontology Id'
manual_list = manual_gene_list.value.split(",")
if manual_list != ['']:
    go_gene_list = list(set(go_gene_list + manual_list))
    
print "Query list of genes:", go_gene_list

Query list of genes: ['ARPC3', 'ARPC1B', 'ARPC1A', 'ARPC5L', 'ACTR3C', 'ACTR3B', 'ARPC5', 'ARPC2', 'ARPC4', 'ACTR3', 'ACTR2']


### Query the IDR to get all the phenotypes associated with the list of genes

In [7]:
# display the results in a table
# set the columns so that all values will be shown
pandas.set_option('display.max_colwidth', -1)

# then display the table
[query_genes_dataframe, screen_to_phenotype_dictionary] = get_phenotypes_for_genelist(session, go_gene_list, organism)
display(HTML(query_genes_dataframe.to_html( escape=False)))

,Entrez,Ensembl,Key,Value,PhenotypeName,PhenotypeAccession,ScreenIds
HELZ2,[85441],[],EntrezID,85441,"[elongated cell phenotype, cell with projections]","[CMPO_0000077, CMPO_0000071]","[1202, 1202]"
ASH2L,[9070],[ENSG00000129691],GeneName,ASH2L,[elongated cell phenotype],[CMPO_0000077],[1202]


---
## Phenotypes to genes query

### Choose a phenotype that we'd like to query
Enter a Phenotype Term Name E.g. elongated cell phenotype

In [6]:
#Phenotype = textbox_widget('elongated cell phenotype', '', 'Phenotype', True)
# ENTER A VALUE IN THE BOX THEN MOVE TO THE NEXT CELL WITHOUT RUNNING THE CELL AGAIN
# THE BOX HAS BE PREPOPULATED WITH A PHENOTYPE VALUE BUT YOU CAN CHANGE THIS

Phenotype = dropdown_widget(
    ['elongated cell phenotype', 'round cell phenotype', 'binuclear cell phenotype', 'increased rate of protein secretion', 'cell response to DNA damage phenotype'],
    'Phenotype: ', True
)

# SELECT THE METHOD THEN MOVE TO THE NEXT CELL WITHOUT RUNNING THE CELL AGAIN



A Jupyter Widget

### We can write a function to query the IDR for the genes associated with a phenotype

In [13]:
def get_genes_for_phenotype(phenotype, conn, sid): 
    args = {
            "name": "Phenotype Term Name",
            "value": phenotype,
            "ns": "openmicroscopy.org/mapr/phenotype",
            "ns2": "openmicroscopy.org/mapr/gene",
            "s_id": sid
        }

    cc = attributes_by_attributes(conn, **args)
    dataframe = pandas.DataFrame.from_dict(cc)
       
    return dataframe


### Then we set up the connection to the IDR

In [14]:
conn = connection()
idr_base_url='https://idr.openmicroscopy.org'

Connected to IDR...


### Then we get a list of all the screens with human data

In [15]:
sid_list = get_organism_screenids(session, organism, idr_base_url)

print ("The IDR identifiers for the screens with human genes are:", sid_list)

('The IDR identifiers for the screens with human genes are:', ['102', '253', '206', '251', '803', '1351', '1202', '1101', '1302', '1251', '1151', '1203', '1204', '1851', '1651', '1652', '1653', '1654', '1801', '1751', '1901', '1952'])


### Go through each screen and find the genes associated with the selected phenotype

In [16]:
phenotype = Phenotype.value
gene_set = set()

for i, sid in enumerate(set(sid_list)):
    df = get_genes_for_phenotype(phenotype, conn, sid)
    progress(i+1, len(set(sid_list)), status='Iterating through screens')
    if df.empty:
        continue

    for x in df.iloc[:, 0]:

        key = x[0]
        value = x[1]

        if key == "Gene Symbol":
            if value != '':
                genesym = value
                gene_set.add(genesym)

gene_list = list(gene_set) 
print gene_list
print "\nThere are", len(gene_list), "unique genes"

['CPM', 'STK16', 'C1orf125', 'PPWD1', 'GCS1_HUMAN', 'ASS1', 'RYK', 'KIR3DX1', 'CYP46A1', 'PTGER2', 'PIK3CG', 'VN1R1', 'ESPL1', 'B3GALTL', 'WEE1', 'ZNF688', 'IFNK', 'PRSS35', 'PPP2R2B', 'PPP2R2A', 'CAMKV', 'CEP110', 'AKAP5', 'C19orf54', 'ARHGAP17', 'CSN3', 'HHIP', 'PHPT1', 'DTL', 'Q8N4N6_HUMAN', 'GPR19', 'EDC4', 'SP2', 'EFTUD2', 'DGCR6L', 'GTF2I', 'TMEM125', 'ARL13B', 'C3orf52', 'KCNK18', 'UCK1', 'SENP5', 'C8orf44', 'NP_001092691.1', 'CHST8', 'CNTFR', 'ARL5A', 'FTL', 'RPS6KA2', 'NP_689919.1', 'CABP7', 'MAP3K2', 'GPX2', 'GPX5', 'USE1', 'GOSR2', 'XP_496076.1', 'GABRR1', 'RCAN3', 'POLG', 'ZWILCH', 'CDC2', 'CDC6', 'RGMA', 'Q8NA17_HUMAN', 'NCOA6', 'TNFAIP1', 'SMC3', 'RBM10', 'SAP30BP', 'TUBB2C', 'LIMD1', 'NP_001007190.1', 'ATOH8', 'SCN11A', 'GPRIN1', 'GRID1', 'EGLN2', 'CHRDL1', 'Q8IV12_HUMAN', 'MCAM', 'BFAR', 'ZNF579', 'CXorf25', 'GJA5', 'CMBL', 'C13orf23', 'TPCN1', 'WDR85', 'CCDC115', 'TBCC', 'MYL6B', 'CRTAP', 'NP_001034202.1', 'EDG8', 'KIAA0649', 'WDR81', 'TXNL4A', 'VRK1', 'PRC1', 'WDR89',

### Now we can perform enrichment analysis on the the gene set associated with the phenotype 
It will find which terms are over-represented using annotations for that gene set.
We are going to use the python package called GSEApy to do this.  It performs gene set enrichment analysis by calling the enrichr online api. See  http://pythonhosted.org/gseapy/run.html for more information about the package, and http://amp.pharm.mssm.edu/Enrichr/ for more information about Enrichr.

### Set which database to query 
You can pick on the list shown on this web page http://amp.pharm.mssm.edu/Enrichr/#stats

Good ones to try are: <br>
* GO_Cellular_Component_2017b
* GO_Biological_Process_2017b
* GO_Molecular_Function_2017b
* Reactome_2016

In [17]:
# EDIT THE NEXT LINE IF YOU WANT TO CHANGE THE DATABASE
databaseToQuery = 'GO_Cellular_Component_2017b'

### Create a function to do the enrichment analysis and then print out the top 5 enriched categories

In [ ]:
def getenrichedterms(df, cutoff=0.05):
    enrichedlist = []
    for GOterm in df.iterrows():
        pvalue = df.loc[int(GOterm[0]),'Adjusted P-value']
        if pvalue <= cutoff:
            term = df.loc[GOterm[0],'Term']
            start1 = term.index('(')+1
            end1 = term.index(')')
            enrichedlist.append(term[start1:end1])
    return enrichedlist

    
print '\033[1m' + "query genes:" + '\033[0m'
enr = gp.enrichr(gene_list, gene_sets=databaseToQuery, cutoff=0.05, no_plot=False)
gsea_results= enr.res2d
gsea_results = gsea_results.sort_values('P-value', ascending=True)
display(gsea_results.head())


query genes:


**Note** 
Information about the values in each column can be found here http://amp.pharm.mssm.edu/Enrichr/help#background

    

<br>
#### When you are completely finished running the notebook close the connection to IDR

In [17]:
conn.close()

This notebook has been modifed from previous notebooks created by Balaji Ramalingam.